In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df = df.fillna('')
test = test.fillna('')

In [6]:
df['total'] = df['title']+' '+df['author']
test['total']=test['title']+' '+test['author']

In [7]:
X = df.drop('label',axis=1)

In [8]:
y=df['label']

In [9]:
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


In [10]:
X


,id,title,author,text,total
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Why the Truth Might Get You Fired Consortiumne...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Rapper T.I.: Trump a ’Poster Child For White S...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Macy’s Is Said to Receive Takeover Approach by...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal..."


In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.4.0'

In [13]:
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [14]:
voc_size = 5000

In [15]:
msg = X.copy()
msg_test = test.copy()

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pmohd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [19]:
for i in range(len(msg)):
  review = re.sub('[^a-zA-Z]',' ',msg['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [20]:
corpus_test = []

In [21]:
for i in range(len(msg_test)):
  review = re.sub('[^a-zA-Z]',' ',msg_test['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)
  

In [22]:
onehot_rep = [one_hot(words,voc_size)for words in corpus]

In [23]:
onehot_rep_test = [one_hot(words,voc_size)for words in corpus_test]

In [24]:
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=25)

In [25]:
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)

In [26]:
model = Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 25, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [27]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)
test_final = np.array(embedded_docs_test)

In [28]:
X_final.shape,y_final.shape,test_final.shape

((20800, 25), (20800,), (5200, 25))

In [29]:
model.fit(X_final,y_final,epochs=25,batch_size=64)

Epoch 1/25
325/325 [==============================] - 14s 27ms/step - loss: 0.3030 - accuracy: 0.8461
Epoch 2/25
325/325 [==============================] - 8s 24ms/step - loss: 0.0168 - accuracy: 0.9952 0s - loss: 0.0167 - accura
Epoch 3/25
325/325 [==============================] - 7s 22ms/step - loss: 0.0077 - accuracy: 0.9979
Epoch 4/25
325/325 [==============================] - 7s 22ms/step - loss: 0.0044 - accuracy: 0.9992
Epoch 5/25
325/325 [==============================] - 7s 22ms/step - loss: 0.0034 - accuracy: 0.9984
Epoch 6/25
325/325 [==============================] - 7s 22ms/step - loss: 0.0020 - accuracy: 0.9991
Epoch 7/25
325/325 [==============================] - 8s 24ms/step - loss: 0.0033 - accuracy: 0.9993
Epoch 8/25
325/325 [==============================] - 8s 24ms/step - loss: 0.0021 - accuracy: 0.9991
Epoch 9/25
325/325 [==============================] - 7s 23ms/step - loss: 0.0012 - accuracy: 0.9996
Epoch 10/25
325/325 [==============================] - 7s 23ms/

In [30]:
y_pred = model.predict_classes(X_final)

C:\Users\pmohd\anaconda3\envs\fakenewspred\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [31]:
pd.DataFrame(y_pred)

,0
0,1
1,0
2,1
3,1
4,1
...,...
20795,0
20796,0
20797,0
20798,1


In [32]:
model.evaluate(X_final,y_final)

650/650 [==============================] - 7s 8ms/step - loss: 5.7887e-06 - accuracy: 1.0000


[5.7886763897840865e-06, 1.0]

In [33]:
model.save('eeeee.h5')